# data load & 전처리

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt

In [1]:
df_train = pd.read_csv('./data/act_train.csv', parse_dates=['date'])
df_people = pd.read_csv('./data/people.csv', parse_dates=['date'])
df_merge = pd.merge(df_train, df_people, how='inner', on='people_id')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [2]:
df_date_x = df_merge['date_x'].copy()
df_date_y = df_merge['date_y'].copy()

df_merge.drop(['date_x', 'date_y'], axis=1, inplace=True)

In [3]:
df_merge = df_merge.replace('type ', '', regex=True).apply(pd.to_numeric, errors='ignore')

In [4]:
df_merge.insert(loc=2 ,column='date_x', value=df_date_x)
df_merge.insert(loc=18 ,column='date_y', value=df_date_x)

#### test data

In [5]:
# df_test = pd.read_csv('./data/act_test.csv', parse_dates=['date'])
# df_merge_test = pd.merge(df_test, df_people, how='inner', on='people_id')
# df_date_x_test = df_merge['date_x'].copy()
# df_date_y_test = df_merge['date_y'].copy()

# df_merge_test.drop(['date_x', 'date_y'], axis=1, inplace=True)

# df_merge_test = df_merge_test.replace('type ', '', regex=True).apply(pd.to_numeric, errors='ignore')

# df_merge_test.insert(loc=2 ,column='date_x', value=df_date_x_test)
# df_merge_test.insert(loc=18 ,column='date_y', value=df_date_y_test)

In [6]:
del df_train, df_people, df_date_x, df_date_y
# del df_test, df_date_x_test, df_date_y_test

In [7]:
df_merge.dtypes

people_id                    object
activity_id                  object
date_x               datetime64[ns]
activity_category             int64
char_1_x                    float64
char_2_x                    float64
char_3_x                    float64
char_4_x                    float64
char_5_x                    float64
char_6_x                    float64
char_7_x                    float64
char_8_x                    float64
char_9_x                    float64
char_10_x                   float64
outcome                       int64
char_1_y                      int64
group_1                      object
char_2_y                      int64
date_y               datetime64[ns]
char_3_y                      int64
char_4_y                      int64
char_5_y                      int64
char_6_y                      int64
char_7_y                      int64
char_8_y                      int64
                          ...      
char_14                        bool
char_15                     

# EDA

In [21]:
for d in ['date_x', 'date_y']:
    print('Start of ' + d + ': ' + str(df_merge[d].min().date()))
    print('  End of ' + d + ': ' + str(df_merge[d].max().date()))
    print('Range of ' + d + ': ' + str(df_merge[d].max() - df_merge[d].min()) + '\n')

Start of date_x: 2022-07-17
  End of date_x: 2023-08-31
Range of date_x: 410 days 00:00:00

Start of date_y: 2022-07-17
  End of date_y: 2023-08-31
Range of date_y: 410 days 00:00:00



In [ ]:
a = df_merge.sample(frac = 0.1).copy()

In [ ]:
a.dtypes

In [ ]:
a = a.replace('type ', '', regex=True).apply(pd.to_numeric, errors='ignore')

In [ ]:
# a.dtypes

In [ ]:
a.head()

In [ ]:
a.columns

In [ ]:
sns.countplot(x='activity_category', hue = 'outcome', data = a)
plt.show()

In [ ]:
# with open('test_test.csv', 'w') as csvfile:
#     fieldnames = ['Id', 'SalePrice']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
#     writer.writeheader()
#     for i in test_test.index:
#         writer.writerow({'Id' : i, 'SalePrice' : test_test.loc[i]})